# CyC-production COVID-19 UKB analyses

## Extract phenotype

COVID-19 phenotype data downloaded from UKB data portal on June 23rd 2021

In [ ]:
library(reshape2)
library(dplyr)
library(tidyr)
library(tidyverse)
library(stringr)
library(survival)
library(fst)

#COVID tests
library(readr)
covid19_result_england <- read_delim("covid19_result_england.txt", 
                                     "\t", escape_double = FALSE, trim_ws = TRUE)
pos_england = subset(covid19_result_england, result==1)$eid

covid19_result_scotland <- read_delim("covid19_result_scotland.txt", 
                                      "\t", escape_double = FALSE, trim_ws = TRUE)
pos_scotland = subset(covid19_result_scotland, result==1)$eid

covid19_result_wales <- read_delim("covid19_result_wales.txt", 
                                   "\t", escape_double = FALSE, trim_ws = TRUE)

pos_wales = subset(covid19_result_wales, result==1)$eid
pos_all = unique(c(pos_england, pos_scotland, pos_wales))

#Hospital-coded
hesin_diag <- read_delim("hesin_diag.txt", 
                         "\t", escape_double = FALSE, trim_ws = TRUE)
metabolic<-c("U071")
hesin_diag = subset(hesin_diag, diag_icd10 %in% metabolic & level==1)
hosp_all = unique(hesin_diag$eid)

#In-hospital mortality
hesin <- read_delim("hesin.txt", 
                         "\t", escape_double = FALSE, trim_ws = TRUE)
hesin = inner_join(hesin, subset(hesin_diag, level==1), by=c('eid','ins_index'))
hesin = subset(hesin, disdest_uni=="11001")

death_in_hospital = unique(hesin$eid)


#Critical care admission
hesin_critical <- read_delim("hesin_critical.txt", 
                             "\t", escape_double = FALSE, trim_ws = TRUE)

library(dplyr)
hesin_critical = inner_join(hesin_critical, subset(hesin_diag, level==1), by=c('eid','ins_index'))
critical_all = unique(hesin_critical$eid)

#Deaths
death_cause <- read_delim("death_cause.txt", 
                          "\t", escape_double = FALSE, trim_ws = TRUE)
death_cause = subset(death_cause, cause_icd10 %in% metabolic & level==1)
death_all = unique(death_cause$eid)
death_all = unique(c(subset(hesin_critical, ccdisdest==6)$eid, death_all, death_in_hospital))

death_and_hospital = intersect(death_all, hosp_all)

#Total cases
total = unique(c(pos_all, hosp_all, critical_all, death_all))
covid = data.frame(eid=total)
covid$hospital = ifelse(covid$eid %in% hosp_all,1,0)
covid$critical = ifelse(covid$eid %in% critical_all,1,0)
covid$death = ifelse(covid$eid %in% death_all,1,0)
covid$death_hospital = ifelse(covid$eid %in% c(death_and_hospital,death_in_hospital),1,0)

## Regression - COVID-19 outcomes against CyC-production PGS

CyC-production PGS was scored as described previously using PLINK2, the polygenic score SNP coefficients will be uploaded to PGS Catalog. Covariates included age, sex, genotyping array, recruitment centre and principal components 1-10 which were computed on an list of LD-pruned SNPs provided by UKB ('ukb_snp_qc.txt')

### Hail/Python script for computing principal components in EUR ancestry cohort

In [ ]:
import hail as hl
import os
import pandas as pd
import subprocess
from gnomad.sample_qc.ancestry import *

#Define memory and CPU availability

tmp = "/mnt/grid/janowitz/rdata_norepl/tmp"

os.environ["SPARK_LOCAL_DIRS"]=tmp
os.environ["PYSPARK_SUBMIT_ARGS"] ="--conf spark.network.timeout=15m --conf spark.executor.heartbeatInterval=10m --conf spark.memory.fraction=1.0 --driver-memory 2880G pyspark-shell"


hl.init(default_reference='GRCh37', master ='local[96]',min_block_size=128, local_tmpdir=tmp, tmp_dir=tmp)


#Pre-process ancestry predictions, merge with phenotype data
ukb_predictions = hl.read_table("ukb_predictions.ht")

ukb_predictions_pd = ukb_predictions.to_pandas()
ukb_predictions_pd = ukb_predictions_pd[["s", "pop"]]
ukb_predictions_pd = ukb_predictions_pd[ukb_predictions_pd['pop'] != "Other"]
print(ukb_predictions_pd['pop'].value_counts(), flush=True)

phenotypes = pd.read_csv('phenotypes.csv')
phenotypes = phenotypes.drop(phenotypes.columns[0], axis=1)
phenotypes = phenotypes.rename(columns={'id': 's'})
phenotypes['s'] = phenotypes.s.astype(int)
ukb_predictions_pd['s'] = ukb_predictions_pd.s.astype(int)

phenotypes = phenotypes.merge(ukb_predictions_pd, how='inner', on='s')

populations = phenotypes['pop'].unique().tolist()
#populations = ["CSA"]

#Add relationship matrix (UKB-provided KING analysis)
rel = hl.import_table('ukbXXXXX_rel_sXXXXX.dat', impute=True,delimiter='\s+', 
                     types={'ID1': hl.tstr, 'ID2': hl.tstr})
rel = rel.select(rel.ID1, rel.ID2, rel.Kinship)

#Import GRCh37 data (filtered to call rate, MAF, no H-W filtering at present)
ukb_gwas = hl.read_matrix_table('ukb_grch37_filtered_forgwas.mt')
ukb_gwas = ukb_gwas.filter_rows(ukb_gwas.variant_qc.AC[1] > 100) #As per Regenie documentation
print(ukb_gwas.count(), flush=True)

#Prune GRCh37 data using SNP list (in_PCA) from marker paper
snp_ukb = hl.import_table('ukb_snp_qc.txt', impute=True, delimiter='\s+')
snp_ukb = snp_ukb.filter(snp_ukb.in_PCA==1)
snp_ukb = snp_ukb.key_by(snp_ukb.rs_id)
ukb_gwas_pruned = ukb_gwas.filter_rows(hl.is_defined(snp_ukb[ukb_gwas.rsid]))

for pop in populations:
    print(pop, flush=True)
    phenotypes['s'] = phenotypes.s.astype(str)
    
    #Subset covariate file, make suitable for BOLT-LMM
    
    subset = phenotypes[phenotypes['pop'] == pop]
    
    frame = { 'FID': subset['s'], 'IID': subset['s'], 'year_of_birth': subset['year_of_birth_f34_0_0'],
             'sex': subset['sex_f31_0_0'],'array': subset['genotype_measurement_batch_f22000_0_0'],
        'centre': subset['uk_biobank_assessment_centre_f54_0_0']}
    
    #Subset GRCH37 version of UK biobank dataset
    
    subset_ht = hl.Table.from_pandas(subset['s'].to_frame(), key='s') 
    ukb_pop_gwas = ukb_gwas.filter_cols(hl.is_defined(subset_ht[ukb_gwas.col_key]))
    ukb_pop_gwas = hl.variant_qc(ukb_pop_gwas)
    ukb_pop_gwas = ukb_pop_gwas.filter_rows(ukb_pop_gwas.variant_qc.AC[1] > 20) #As per Regenie documentation
    ukb_pop_gwas_pruned = ukb_gwas_pruned.filter_cols(hl.is_defined(subset_ht[ukb_gwas_pruned.col_key]))

    #Run PCA on all filtered variants, per population, removing relateds subjects per population
    
    rel_sub = rel.filter(hl.is_defined(subset_ht[rel.ID1]) & hl.is_defined(subset_ht[rel.ID2]))
    pairs = rel_sub.filter(rel_sub['Kinship'] > 0.0442) #Third degree relatives per KING documentation
    related_samples_to_remove = hl.maximal_independent_set(pairs.ID1, pairs.ID2, False)
    
    print("Related + ", pop, related_samples_to_remove.count(), flush=True)
    
    _, scores_pca_ref, _ = run_pca_with_relateds(ukb_pop_gwas_pruned, related_samples_to_remove, 
                                                               n_pcs=10, autosomes_only=True)
    
    scores_pca_ref = scores_pca_ref.transmute(**{f'PC{i}': scores_pca_ref.scores[i - 1] for i in range(1, 11)})
    scores_pd = scores_pca_ref.to_pandas()
    scores_pd = scores_pd.rename(columns={'s': 'IID'})
    
    #Add PCA results to covariates file, then save
    
    covariates = covariates.merge(scores_pd, left_on='IID', right_on='IID')
    folder = 'PATH' + pop + '/covariates.tsv'
    covariates.to_csv(folder, sep="\t", index=False)


### Regression (R script)

In [ ]:
library(readr)
library(dplyr)

scale2 <- function(x, na.rm = TRUE) (x - mean(x, na.rm = na.rm)) / sd(x, na.rm)
PGS_1e4_RAPIDO <- read_table2("UKB_UKB380_PGS_inner.sscore")



names(PGS_1e4_RAPIDO)[2]<-"id"
PGS_1e4_RAPIDO$SCORE = PGS_1e4_RAPIDO$SCORE1_AVG

#Import additional variables
populations <- read_csv("populations.csv")
PGS_1e4_RAPIDO = subset(PGS_1e4_RAPIDO, id %in% subset(populations, pop=="EUR")$s)

#Filter to cohort of patients ('validation cohort') who were not used for construct of the polygenic score
validation_cohort <- read_delim("validation_cohort.tsv", 
                                "\t", escape_double = FALSE, trim_ws = TRUE)

covariates <- read_table2("covariates_eur.tsv")
covariates = covariates[,-4]
names(covariates)[2]<-"id"


PGS_1e4_RAPIDO = subset(PGS_1e4_RAPIDO, id %in% validation_cohort$s)
PGS_1e4_RAPIDO = inner_join(PGS_1e4_RAPIDO, covariates, by="id")

#Remove any duplicated cystatin measurements, pick earliest measurement

#Normalize score
PGS_1e4_RAPIDO$score_scale = scale2(PGS_1e4_RAPIDO$SCORE)
PGS_1e4_RAPIDO$score_decile <- as.factor(ntile(PGS_1e4_RAPIDO$SCORE, 10))
PGS_1e4_RAPIDO$score_quartile <- as.factor(ntile(PGS_1e4_RAPIDO$SCORE, 4))


names(PGS_1e4_RAPIDO)[2]='eid'
covidx <- inner_join(covid, PGS_1e4_RAPIDO, by='eid')
m<-match(covidx$eid, survival$id)
covidx$sex = ifelse(survival$sex[m]=="Male",1,0)
covidx$bmi = survival$bmi[m]

covidx$control = ifelse(covidx$hospital==0 & covidx$critical==0 & covidx$death==0,1,0)


comb = data.frame()
mylogit = glm(formula = hospital ~ score_scale + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + year_of_birth + centre + array, family = "binomial", data=subset(covidx, hospital ==1 | control==1))
add = data.frame(comparison="hospitalization",  HR = exp(mylogit$coefficients[2]), lower = exp(confint(mylogit, level=0.95)[2,1]),
                 upper = exp(confint(mylogit, level=0.95)[2,2]), p_value=summary(mylogit)$coefficients[2,4])
comb = rbind(comb, add)

mylogit = glm(formula = critical ~ score_scale + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + year_of_birth + centre + array, family = "binomial", data=subset(covidx, critical ==1 | control==1))
add = data.frame(comparison="critical care",  HR = exp(mylogit$coefficients[2]), lower = exp(confint(mylogit, level=0.95)[2,1]),
                 upper = exp(confint(mylogit, level=0.95)[2,2]), p_value=summary(mylogit)$coefficients[2,4])
comb = rbind(comb, add)

mylogit = glm(formula = death ~ score_scale + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + year_of_birth + centre + array, family = "binomial",data=subset(covidx, death ==1 | control==1))
add = data.frame(comparison="death",  HR = exp(mylogit$coefficients[2]), lower = exp(confint(mylogit, level=0.95)[2,1]),
                 upper = exp(confint(mylogit, level=0.95)[2,2]), p_value=summary(mylogit)$coefficients[2,4])
comb = rbind(comb, add)

comb %>% dplyr::mutate(comparison = factor(comparison, levels = c("death", "critical care","hospitalization"))) %>% ggplot(aes(x = comparison,y = HR, ymin = lower, ymax = upper ))+
  geom_pointrange(aes(col=comparison))+
  geom_hline(aes(fill=comparison),yintercept =1, linetype=2)+
  xlab('Group')+ ylab("Odds Ratio (95% Confidence Interval)")+
  geom_errorbar(aes(ymin=lower, ymax=upper,col=comparison),width=0.5,cex=1)+ 
  theme(plot.title=element_text(size=16,face="bold"),
        axis.text.y=element_blank(),
        axis.ticks.y=element_blank(),
        axis.text.x=element_text(face="bold"),
        axis.title=element_text(size=12,face="bold"),
        strip.text.y = element_text(hjust=0,vjust = 1,angle=180,face="bold"))+
  coord_flip()+theme_bw()+xlab("")+ theme(legend.position = "none") 

### Population controls analysis (R script)

In [ ]:
use = PGS_1e4_RAPIDO
m<-match(use$eid, survival$id)
use$sex = ifelse(survival$sex[m]=="Male",1,0)
use$bmi = survival$bmi[m]


controls = subset(use, !(eid %in% covid$eid))
controls$stat = 0
cases = subset(use, eid %in% subset(covid, critical==1)$eid)
cases$stat = 1

analysis = rbind(controls, cases)
mylogit = glm(formula = stat ~ score_scale + bmi + sex + PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + year_of_birth + centre + array, family = "binomial", data=analysis)
summary(mylogit)
add = data.frame(comparison="hospitalization",  HR = exp(mylogit$coefficients[2]), lower = exp(confint(mylogit, level=0.95)[2,1]),
                 upper = exp(confint(mylogit, level=0.95)[2,2]))
print(add)

## Charite Hospital C2 ratio analysis

In [ ]:
library(dplyr)
library(ggplot2)
library(readr)
library(readxl)
data <- read_delim("data.txt", 
                   "\t", escape_double = FALSE, trim_ws = TRUE)

metadata <- read_delim("metadata.txt", 
                       "\t", escape_double = FALSE, trim_ws = TRUE)

new_data <- read_excel("20220209_data_long_upd.xlsx", sheet="minimal")
new_data$patient2 = paste('00',new_data$patient,sep='')


process = as.data.frame(t(data[,-1]))
names(process) = data$Features

process$patient<-row.names(process)

names(metadata)[1]<-"patient"
process = inner_join(metadata, process, by="patient")
process=process[,c(1,2:19,49,50, 96, 51, 97, 84, 98, 99,53,36)]

m<-match(process$patient,new_data$patient)
m2<-match(process$patient,new_data$patient2)
m3=coalesce(m, m2)

process$creatinine_new=new_data$`Creatinine, mg/dl`[m3]
process$crp_new=new_data$`CRP (clinical), mg/l`[m3]
process$bmi_new=new_data$BMI[m3]



process$`Creatinine, mg/dl`=coalesce(process$`Creatinine, mg/dl`, process$creatinine_new)

process = process %>% arrange(SP.Date) %>% group_by(Id) %>% tidyr::fill(`Creatinine, mg/dl`, .direction="downup") #Impute with most recent value
process = process %>% arrange(SP.Date) %>% group_by(Id) %>% tidyr::fill(`CRP (clinical), mg/l`, .direction="downup") #Impute with most recent value
process = process %>% arrange(SP.Date) %>% group_by(Id) %>% tidyr::fill(CST3, .direction="downup") #Impute with most recent value


process$`Creatinine, mg/dl`[is.nan(process$`Creatinine, mg/dl`)] <- NA

process = subset(process, is.na(`Creatinine, mg/dl`)==FALSE)

process$outcome_date = process$SP.Date - process$I.Days.Delta.Outcome

process$os_days = NA

for(i in 1:nrow(process)) {
  
  row = process[i,]
  if(row$I.Died ==1) {process$os_days[i] = row$outcome_date - row$SP.Date}
  if(row$I.Died ==0) {process$os_days[i] = as.Date("2020-12-30") - row$SP.Date}
  
}

process$os_stat = process$I.Died

library(nephro)
library(tidyr)

process$ethnicity<-0

process = process %>% tidyr::drop_na(c("F.Sex","Creatinine, mg/dl", "I.Age","CST3", "BMI")) %>% mutate(F.Sex = ifelse(F.Sex=="MaNNLICH",1,0)) %>% mutate(egfr_creatinine=CKDEpi.creat(`Creatinine, mg/dl`,F.Sex, I.Age, ethnicity),egfr_cystatin = CKDEpi.cys((CST3/300),F.Sex, I.Age))

model=lm(egfr_cystatin ~ egfr_creatinine, data=process)
process$cortiscore = -resid(model)

process$ratio = process$egfr_creatinine / process$egfr_cystatin
ggplot(process, aes(x=as.factor(I.After.Dexa), y=cortiscore))+geom_jitter(width=0.1)


library("survival")
library("survminer")
library(finalfit)

process$cortistat = ifelse(process$ratio >1, 1, 0)

#process = process %>% group_by(Id) %>% slice_min(order_by=SP.Date, n=1)

process = process  %>% mutate(F.Sex = as.factor(F.Sex), CRP = `CRP (clinical), mg/l`, charlson = `Charlson score`)



library(coxme)

m2 <- coxph(Surv(os_days, os_stat) ~ I.Age + F.Sex + BMI + cortistat + CRP + charlson + cluster(Id), 
            data = process)


m3 <- coxph(Surv(os_days, os_stat) ~ cortistat + cluster(Id), 
            data = process)


summary(m2)
summary(m3)